# Using `sklearn` Pipelines for Custom Data Transformations

The purpose of this notebook is to demonstrate how to perform custom data transformations and analyses using `sklearn` Pipelines.  The relevant Python pseudo-code looks something like this:

```python
# Let's write a custom data transformer as an example!
from sklearn.base import BaseEstimator, TransformerMixin

# The code looks something like this:
class MyMovingAverageAdder(BaseEstimator, TransformerMixin):
    """
    Custom data transformer, intended to be used as part of a `sklearn` Pipeline.
    
    See file `./technical_analysis.py` contained in this directory for more details.
    """
    ...

```

In the code snippet shown above, there are two important points: 

2. The custom class inherits from `sklearn.base.BaseEstimator` so that we may gain methods `get_params()` and `set_params()`, which are useful in hyperparameter optimization, for example.  
1. The custom class inherits from `sklearn.base.TransformerMixin` so that we may gain methods `fit()`, `transform()`, and `fit_transform()`, which are useful in creating data-transformation pipelines, for example.

More information can be found in [sklearn.base](https://scikit-learn.org/stable/modules/classes.html?highlight=sklearn%20base#module-sklearn.base).


In [1]:
# Autoreload modules we're working on so that changes propagate to this notebook
%load_ext autoreload
%autoreload 1
%aimport technical_analysis

In [2]:
# Import the usual suspects....
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline


import pandas_datareader as pdr
import datetime as dt
import quandl


import pandas as pd
import numpy as np
import requests
from termcolor import colored as cl
from math import floor
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20, 10)
plt.style.use('fivethirtyeight')

In [3]:
# Here is our custom code!
from technical_analysis import *
from technical_analysis_m import *

## Generate Synthetic Test Data

In [4]:
# Define "percent" and other units & shorthands (for clarity below)
percent = 0.01
usd = 1.0  # $US

# Generate some test data
returns_mean = 0.0 * percent
returns_std = 1.0 * percent
num_business_days = 252
initial_price = 200 * usd
returns = returns_mean + (returns_std * np.random.randn(num_business_days))
timeseries = initial_price * (1 + pd.DataFrame(data=returns, columns=['close'])).cumprod()
X = timeseries.copy()  # shorthand

# Display test data
X.head()

,close
0,197.969613
1,196.976651
2,199.819100
3,201.078700
4,203.770457


In [5]:
# ticker0 = quandl.get("EOD/AAPL", authtoken="YA4KnPAgb7_ouvkkyxKs")
# # print(ticker0.size )
# ticker1 = ticker0.copy()
# ticker1.columns= ticker0.columns.str.strip().str.lower()
# ticker = ticker1.iloc[-30:]
# print(ticker.size)
# ticker.tail()

In [6]:
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import os
import pandas as pd

# https://algotrading101.com/learn/alpaca-trading-api-guide/
load_dotenv("C:\\Users\\thebe\\Fintech\\API_KEYS.env")

# authentication and connection details
# print(os.getenv("ALPACA_API_KEY"))
api_key = os.getenv("ALPACA_API_KEY")
api_secret = os.getenv("ALPACA_SECRET_KEY")
base_url = 'https://paper-api.alpaca.markets'

# instantiate REST API
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')

# obtain account information
account = api.get_account()
# print(account)

In [7]:

today = pd.Timestamp("2020-07-14", tz="America/New_York").isoformat()

# tickers = ["FB", "TWTR", 'AAPL']
tickers = ['AAPL']
# tickers =["FB"]

timeframe = "1D"



start = pd.Timestamp("2017-07-2", tz="America/New_York").isoformat()
end = pd.Timestamp("2021-07-2", tz="America/New_York").isoformat()

# Get current closing prices for FB and TWTR
ticker0 = api.get_barset(
    tickers,
    timeframe,
    start = start,
    end = end,
).df

ticker = ticker0.copy()
ticker.columns= ticker0["AAPL"].columns.str.strip().str.lower()


# Display sample data
ticker.head(5)

,open,high,low,close,volume
time,,,,,
2021-02-10 00:00:00-05:00,136.48,136.99,134.4000,135.36,66856066
2021-02-11 00:00:00-05:00,135.90,136.39,133.7700,135.12,59377028
2021-02-12 00:00:00-05:00,134.35,135.53,133.6921,135.38,54861442
2021-02-16 00:00:00-05:00,135.49,136.01,132.7900,133.16,72596290
2021-02-17 00:00:00-05:00,131.25,132.22,129.4700,130.84,92298227


## Build Custom Transformation Pipeline

In [11]:
# Build an `sklearn.pipeline.Pipeline`
pipeline = Pipeline([
    ('sma20', MovingAverageAdder(ma_type='sma', window=20)),  # simple moving average (SMA)
    ('ema20', MovingAverageAdder(ma_type='ema', window=20)),  # exponential moving average (EMA)
    ('dema20', MovingAverageAdder(ma_type='dema', window=20)),  # double exponential moving average (DEMA)
    ('tema20', MovingAverageAdder(ma_type='tema', window=20)),  # triple exponential moving average (TEMA)
    ('stochastic oscillator', MarcusIndicators(ma_type='stochastic oscillator', window=20)),
    ('ichimoku cloud', MarcusIndicators(ma_type='ichimoku cloud', window=20)),
    ('fibonacci retracement', MarcusIndicators(ma_type='fibonacci retracement', window=20)),
    ('bollinger_band', BennyIndicators(indicator='bollinger_bands', bb_window=20)),
    ('rsi', BennyIndicators(indicator='rsi', lookback=14)),
    ('macd', BennyIndicators(indicator='macd', slow=26, fast=12, smooth=9))
])

[autoreload of technical_analysis failed: Traceback (most recent call last):
  File "C:\Users\thebe\anaconda\envs\algotrading\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\thebe\anaconda\envs\algotrading\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\Users\thebe\anaconda\envs\algotrading\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\Users\thebe\anaconda\envs\algotrading\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_

## Transform Data

In [12]:
# Transform the input data
# X_transformed = pipeline.fit_transform(X)

In [13]:
ticker_transformed = pipeline.fit_transform(ticker)

Level Percentage	 Price ($)
00.0%		 139.96
23.6%		 134.39984
38.2%		 130.96008
50.0%		 128.18
61.8%		 125.39992000000001
100.0%		 116.4


ValueError: Wrong number of items passed 3, placement implies 1

In [ ]:
# X_transformed.plot()

In [ ]:
ticker_transformed.plot()

In [ ]:
# X_transformed.tail()

In [ ]:
ticker_transformed.tail()

## Plot Results

In [ ]:
# Plot the results
#ax = X_transformed.plot(linewidth=1.5)
# ax = X_transformed.rename(columns={'close': 'close (input)'}).plot()
# ax.set_title('Stock Price vs. Time (Synthetic Data)')
# ax.set_xlabel('Number of Business Days')
# ax.set_ylabel('Stock Price ($US)')
# ax.legend(title='Pipeline Results', ncol=1)
# plt.show()

In [ ]:
# Plot the results
ax = ticker_transformed.plot(linewidth=1.5)
ax = ticker_transformed.rename(columns={'close': 'close (input)'}).plot()
ax.set_title('Stock Price vs. Time (Synthetic Data)')
ax.set_xlabel('Number of Business Days')
ax.set_ylabel('Stock Price ($US)')
ax.legend(title='Pipeline Results', ncol=1)
# plt.show()

In [ ]:
# from matplotlib import pyplot as plt
plot_stoch('AAPL', ticker_transformed['close'], ticker_transformed['%K'],ticker_transformed['%D'])

In [ ]:
buy_price, sell_price, stoch_signal = implement_stoch_strategy(ticker_transformed['close'], ticker_transformed['%K'], ticker_transformed['%D'])
plot_sell_buy(ticker_transformed, buy_price, sell_price, stoch_signal)

# Ichimoku Cloud Plot


In [ ]:
kijun_sen_plot(ticker_transformed,kijun_window)

In [ ]:
tenkan_sen_plot(ticker_transformed, tenkan_window )

In [ ]:
Senkou_Span_plot(ticker_transformed, senkou_span_b_window)

In [ ]:
Chikou_plot(ticker_transformed, chikou_shift)

In [ ]:
plot_ichimoku(ticker_transformed,  view_limit=100)

# Fibonacci Retracement Plot

In [ ]:

fib_retracement_plot(
    ticker_transformed,
    ticker_transformed['fib_close_min'][0],
    ticker_transformed['fib_level_1'][0],
    ticker_transformed['fib_level_2'][0], 
    ticker_transformed['fib_level_3'][0],
    ticker_transformed['fib_level_4'][0],
    ticker_transformed['fib_close_max'][0])